In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import math

import pandas as pd
from sklearn.model_selection import train_test_split

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [2]:
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe):
        self.X = torch.tensor(dataframe.iloc[:, :3].values, dtype=torch.float32)  # Input: [Batch, 3]
        self.Y = torch.tensor(dataframe.iloc[:, 3:].values, dtype=torch.float32).unsqueeze(-1) # Output: [Batch, 255, 1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [3]:
# Positional Encoding for Time Steps
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)  # Add batch dimension

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :].to(x.device)

# Transformer Model
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward):
        super(TransformerTimeSeriesModel, self).__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
 
        # Input Encoder (maps input to d_model size)
        self.encoder = nn.Linear(input_dim, d_model)  # (Batch, 3) -> (Batch, d_model)
        
        # Project input to match the sequence length
        self.expand_input = nn.Linear(d_model, seq_length * d_model)  # (Batch, d_model) -> (Batch, seq_length * d_model)
        
        # Target embedding for decoder input
        self.target_embedding = nn.Linear(1, d_model)  # New embedding layer for target sequence
  
        # Positional Encoding for Time Steps
        self.pos_encoder = PositionalEncoding(d_model, seq_length)
        
        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        # Final Output Layer
        self.output_layer = nn.Linear(d_model, output_dim)  # (Batch, 255, d_model) -> (Batch, 255, 1)

    def forward(self, x, target_seq):
        # x: Input features [Batch, 3]
        # target_seq: Target sequence for teacher forcing [Batch, 255, 1]
        
        # Encode input features
        encoded_input = self.encoder(x)  # [Batch, d_model]
        
        # Expand input to match sequence length
        expanded_input = self.expand_input(encoded_input)  # [Batch, seq_length * d_model]
        expanded_input = expanded_input.view(-1, self.seq_length, self.d_model)  # Reshape to [Batch, 255, d_model]
        
        # Add Positional Encoding
        expanded_input = self.pos_encoder(expanded_input)
        
        # Process the target sequence through the same encoding pipeline
  #      target_embeddings = self.encoder(target_seq)
  #      target_embeddings = nn.Linear(1, d_model)(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.target_embedding(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.pos_encoder(target_embeddings)
        
        # Decode sequence
        output = self.transformer_decoder(
            tgt=target_embeddings, memory=expanded_input
        )  # Output shape: [Batch, 255, d_model]
        
        # Map to output dimensions
        predictions = self.output_layer(output)  # [Batch, 255, 1]
        return predictions

In [4]:
def train_model(model, dataloader, optimizer, loss_fn, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch in dataloader:
            x, y = batch  # x: [Batch, N], y: [Batch, T]
            x, y = x.to(device), y.to(device)
            
            # Prepare target for teacher forcing
            target_seq = y 
            #target_seq = y[:, :-1]  # All except last time step
            #actual = y[:, 1:]       # All except first time step
            
            # Forward pass
            output = model(x, target_seq)
            loss = loss_fn(output, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [6]:
# Load the CSV file
data_input = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/epsteinCV_inputs.csv", sep=" ", header=None)
data_output = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/epsteinCV_outputs_active.csv", header=None)
data = pd.concat([data_input, data_output], axis=1)
# Split the data into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the validation set to a new CSV file
valid_data.to_csv("validation_set.csv", index=False)

In [11]:
train_data.head()

,0,1,2,0,1,2,3,4,5,6,...,242,243,244,245,246,247,248,249,250,251
3547,0.776554,0.054693,0.033034,0,591,694,600,495,382,287,...,13,4,2,4,7,9,6,10,2,11
34861,0.982732,0.015491,0.216774,0,1085,1238,1215,1196,1171,1154,...,846,841,858,864,863,853,861,862,855,861
18274,0.974610,0.019119,0.495488,0,877,1122,1104,1080,1051,1033,...,716,712,718,719,715,716,714,732,726,715
33070,0.860337,0.124194,0.152559,0,348,258,56,3,0,0,...,3,1,2,3,6,1,1,4,3,2
29702,0.619412,0.377743,0.037252,0,28,0,0,0,1,0,...,1,0,0,0,0,2,0,1,1,0


In [7]:
# Load your DataFrame (assuming it's named `df`)
dataset = TimeSeriesDataset(train_data)

In [12]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [13]:
# Model parameters
input_dim = 3      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 252   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)


In [14]:
# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 40  # Adjust based on dataset size and performance
train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

Epoch 1/40, Loss: 4665.96337890625
Epoch 2/40, Loss: 1847.5054931640625
Epoch 3/40, Loss: 136.32862854003906
Epoch 4/40, Loss: 675.1090087890625
Epoch 5/40, Loss: 1095.3199462890625
Epoch 6/40, Loss: 74635.546875
Epoch 7/40, Loss: 998.234375
Epoch 8/40, Loss: 18976.77734375
Epoch 9/40, Loss: 519283.0625
Epoch 10/40, Loss: 150936.609375
Epoch 11/40, Loss: 882.1312866210938
Epoch 12/40, Loss: 21653.677734375
Epoch 13/40, Loss: 258784.78125
Epoch 14/40, Loss: 202.67538452148438
Epoch 15/40, Loss: 472110.84375
Epoch 16/40, Loss: 224.34120178222656
Epoch 17/40, Loss: 218.5352020263672
Epoch 18/40, Loss: 19627.830078125
Epoch 19/40, Loss: 700.366943359375
Epoch 20/40, Loss: 136.22044372558594
Epoch 21/40, Loss: 124.08040618896484
Epoch 22/40, Loss: 105.12836456298828
Epoch 23/40, Loss: 2429.727783203125
Epoch 24/40, Loss: 203.0005645751953
Epoch 25/40, Loss: 297.48150634765625
Epoch 26/40, Loss: 20931.888671875
Epoch 27/40, Loss: 376.426513671875
Epoch 28/40, Loss: 653303.5
Epoch 29/40, Loss